# Import MN Poverty Data from the Census
  

In [23]:
import pandas as pd
import requests
from pprint import pprint
from census import Census
from api import geoapify_key, weather_api_key, census_key


## Import the data

In [24]:
#Pull cencus data for poverty
#Years earlier than 2010 or later than 2021 did not pull. So using these 11 years.

#variables:
years = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
poverty_data = []

#Loop through each year and pull in poverty data, later it will be filtered for the state
print("--------------------------")
for year in years:
    #census api definitions below from: https://api.census.gov/data/2017/acs/acs5/groups/B17001.html
    c = Census(census_key, year)
    state_county_data = c.acs5.get(
        ("B17001_001E",
         "B17001_002E",
         "B17001_017E",
         "B17001_018E",
         "B17001_019E",
         "B17001_020E",
         "B17001_021E",
         "B17001_022E",
         "B17001_023E",
         "B17001_024E",
         "B17001_025E",
         "B17001_026E",
         "B17001_027E",
         "B17001_028E",
         "B17001_029E",
         "B17001_030E",
         "B17001_003E",
         "B17001_004E",
         "B17001_005E",
         "B17001_006E",
         "B17001_007E",
         "B17001_008E",
         "B17001_009E",
         "B17001_010E",
         "B17001_011E",
         "B17001_012E",
         "B17001_013E",
         "B17001_014E",
         "B17001_015E",
         "B17001_016E",
         "B01003_001E"),
        {'for': 'county:*'},
    )

    print(f"Fetching Data for {year}")

    # Convert to DataFrame
    df = pd.DataFrame(state_county_data)
    df['year'] = year
    poverty_data.append(df)
print("--------------------------")
# Concatenate the list of DataFrames into a single DataFrame
merged_df = pd.concat(poverty_data, ignore_index=True)

# Print the merged DataFrame
print(merged_df)

--------------------------
Fetching Data for 2010
Fetching Data for 2011
Fetching Data for 2012
Fetching Data for 2013
Fetching Data for 2014
Fetching Data for 2015
Fetching Data for 2016
Fetching Data for 2017
Fetching Data for 2018
Fetching Data for 2019
Fetching Data for 2020
Fetching Data for 2021
--------------------------
       B17001_001E  B17001_002E  B17001_017E  B17001_018E  B17001_019E  \
0          44247.0      21672.0      11763.0        916.0        187.0   
1         246260.0      53738.0      28239.0       3470.0        608.0   
2           9268.0       1709.0       1044.0        106.0         20.0   
3          13853.0       1664.0        694.0         92.0          0.0   
4         389514.0      54276.0      28534.0       3094.0        772.0   
...            ...          ...          ...          ...          ...   
38641      54242.0      23520.0      12739.0        723.0         90.0   
38642       8317.0       4425.0       2278.0        146.0         64.0   
3864

In [25]:
merged_df.head()

,B17001_001E,B17001_002E,B17001_017E,B17001_018E,B17001_019E,B17001_020E,B17001_021E,B17001_022E,B17001_023E,B17001_024E,...,B17001_011E,B17001_012E,B17001_013E,B17001_014E,B17001_015E,B17001_016E,B01003_001E,state,county,year
0,44247.0,21672.0,11763.0,916.0,187.0,801.0,741.0,184.0,384.0,1222.0,...,1135.0,1060.0,994.0,868.0,697.0,644.0,44631.0,72,091,2010
1,246260.0,53738.0,28239.0,3470.0,608.0,3841.0,1758.0,488.0,986.0,3917.0,...,3243.0,2399.0,1616.0,1131.0,588.0,433.0,250699.0,06,047,2010
2,9268.0,1709.0,1044.0,106.0,20.0,107.0,100.0,69.0,30.0,107.0,...,102.0,36.0,81.0,124.0,21.0,13.0,9605.0,06,049,2010
3,13853.0,1664.0,694.0,92.0,0.0,122.0,14.0,0.0,0.0,165.0,...,206.0,111.0,34.0,73.0,13.0,0.0,13905.0,06,051,2010
4,389514.0,54276.0,28534.0,3094.0,772.0,3418.0,1494.0,424.0,1107.0,3552.0,...,3457.0,2962.0,1907.0,1727.0,708.0,474.0,407435.0,06,053,2010


In [26]:
#filter to show MN only in new DF
minnesota_data_raw = merged_df.loc[merged_df['state'] == '27']

In [27]:
#Rename columns and cleanup data
minnesota_data = minnesota_data_raw.rename(columns={
                      "B17001_001E" : "Sample Population",
                      "B17001_002E" : "Population Poverty",
                      "B17001_017E" : "Females Poverty",
                      "B17001_018E" : "Females < 5 Poverty",
                      "B17001_019E" : "Females 5 Poverty",
                      "B17001_020E" : "Females 6 to 11 Poverty",
                      "B17001_021E" : "Females 12 to 14 Poverty",
                      "B17001_022E" : "Females 15 Poverty",
                      "B17001_023E" : "Females 16 to 17 Poverty",
                      "B17001_024E" : "Females 18 to 24 Poverty",
                      "B17001_025E" : "Females 25 to 34 Poverty",
                      "B17001_026E" : "Females 35 to 44 Poverty",
                      "B17001_027E" : "Females 45 to 54 Poverty",
                      "B17001_028E" : "Females 55 to 64 Poverty",
                      "B17001_029E" : "Females 65 to 74 Poverty",
                      "B17001_030E" : "Females 75 > Poverty",
                      "B17001_003E" : "Males Poverty",
                      "B17001_004E" : "Males < 5 Poverty",
                      "B17001_005E" : "Males 5 Poverty",
                      "B17001_006E" : "Males 6 to 11 Poverty",
                      "B17001_007E" : "Males 12 to 14 Poverty",
                      "B17001_008E" : "Males 15 Poverty",
                      "B17001_009E" : "Males 16 to 17 Poverty",
                      "B17001_010E" : "Males 18 to 24 Poverty",
                      "B17001_011E" : "Males 25 to 34 Poverty",
                      "B17001_012E" : "Males 35 to 44 Poverty",
                      "B17001_013E" : "Males 45 to 54 Poverty",
                      "B17001_014E" : "Males 55 to 64 Poverty",
                      "B17001_015E" : "Males 65 to 74 Poverty",
                      "B17001_016E" : "Males 75 > Poverty",
                      "B01003_012E" : "Total Population", 
                      "state" : "state code",
                      "county" : "county code"
                      })
#Create a column for the state
minnesota_data.insert(32, 'state', 'MN')



In [28]:
minnesota_data['Children in Poverty'] = minnesota_data['Females < 5 Poverty'] + minnesota_data['Females 5 Poverty'] \
    + minnesota_data['Females 6 to 11 Poverty'] + minnesota_data['Females 12 to 14 Poverty'] \
    + minnesota_data['Females 15 Poverty'] + minnesota_data['Females 16 to 17 Poverty'] \
    + minnesota_data['Males < 5 Poverty'] + minnesota_data['Males 5 Poverty'] \
    + minnesota_data['Males 6 to 11 Poverty'] + minnesota_data['Males 12 to 14 Poverty'] \
    + minnesota_data['Males 15 Poverty'] + minnesota_data['Males 16 to 17 Poverty'] 
  
minnesota_data['Seniors in Poverty'] = minnesota_data['Females 55 to 64 Poverty'] + minnesota_data['Females 65 to 74 Poverty'] \
    + minnesota_data['Females 75 > Poverty'] + minnesota_data['Males 55 to 64 Poverty'] \
    + minnesota_data['Males 65 to 74 Poverty'] + minnesota_data['Males 75 > Poverty']

In [29]:
#need to determine per county poverty percentage for each year

minnesota_data['Poverty %'] = (minnesota_data['Population Poverty'] / minnesota_data['Sample Population']) *100
minnesota_data['Female Poverty %'] = (minnesota_data['Females Poverty'] / minnesota_data['Sample Population']) *100
minnesota_data['Male Poverty %'] = (minnesota_data['Males Poverty'] / minnesota_data['Sample Population']) *100
minnesota_data['Childhood Poverty %'] = (minnesota_data['Children in Poverty'] / minnesota_data['Sample Population']) *100
minnesota_data['Senior Poverty %'] = (minnesota_data['Seniors in Poverty'] / minnesota_data['Sample Population']) *100



In [30]:
#Create a column for the counties using the county codes
county_mapping = {'001': 'Aitkin', '003': 'Anoka', '005': 'Becker', '007': 'Beltrami', '009': 'Benton',
                  '011': 'Big Stone', '013': 'Blue Earth', '015': 'Brown', '017': 'Carlton', '019': 'Carver',
                  '021': 'Cass', '023': 'Chippewa', '025': 'Chisago', '027': 'Clay', '029': 'Clearwater',
                  '031': 'Cook', '033': 'Cottonwood', '035': 'Crow Wing', '037': 'Dakota', '039': 'Dodge',
                  '041': 'Douglas', '043': 'Faribault', '045': 'Fillmore', '047': 'Freeborn', '049': 'Goodhue',
                  '051': 'Grant', '053': 'Hennepin', '055': 'Houston', '057': 'Hubbard', '059': 'Isanti',
                  '061': 'Itasca', '063': 'Jackson', '065': 'Kanabec', '067': 'Kandiyohi', '069': 'Kittson',
                  '071': 'Koochiching', '073': 'Lac qui Parle', '075': 'Lake', '077': 'Lake of the Woods', '079': 'Le Sueur',
                  '081': 'Lincoln', '083': 'Lyon', '085': 'McLeod', '087': 'Mahnomen', '089': 'Marshall',
                  '091': 'Martin', '093': 'Meeker', '095': 'Mille Lacs', '097': 'Morrison', '099': 'Mower',
                  '101': 'Murray', '103': 'Nicollet', '105': 'Nobles', '107': 'Norman', '109': 'Olmsted',
                  '111': 'Otter Tail', '113': 'Pennington', '115': 'Pine', '117': 'Pipestone', '119': 'Polk',
                  '121': 'Pope', '123': 'Ramsey', '125': 'Red Lake', '127': 'Redwood', '129': 'Renville',
                  '131': 'Rice', '133': 'Rock', '135': 'Roseau', '137': 'St. Louis', '139': 'Scott',
                  '141': 'Sherburne', '143': 'Sibley', '145': 'Stearns', '147': 'Steele', '149': 'Stevens',
                  '151': 'Swift', '153': 'Todd', '155': 'Traverse', '157': 'Wabasha', '159': 'Wadena',
                  '161': 'Waseca', '163': 'Washington', '165': 'Watonwan', '167': 'Wilkin', '169': 'Winona',
                  '171': 'Wright', '173': 'Yellow Medicine'
}

#Insert the dictionary of county_mapping into a new column for the county
print(minnesota_data.shape)
minnesota_data.insert(34, 'county', "")
print(minnesota_data.columns)

minnesota_data['county'] = minnesota_data['county code'].map(county_mapping)

#Create a column for category of Rural or Urban Based on OMB Statistical Data. 
#These categories are from 2003. Newer categories for 2020 have not been finalized. 
county_type_omb = {'Aitkin': 'Rural', 'Anoka': 'Urban', 'Becker': 'Rural', 'Beltrami': 'Rural', 'Benton': 'Urban',
                  'Big Stone': 'Rural', 'Blue Earth': 'Rural', 'Brown': 'Rural', 'Carlton': 'Urban', 'Carver': 'Urban',
                  'Cass': 'Rural', 'Chippewa': 'Rural', 'Chisago': 'Urban', 'Clay': 'Urban', 'Clearwater': 'Rural',
                  'Cook': 'Rural', 'Cottonwood': 'Rural', 'Crow Wing': 'Rural', 'Dakota': 'Urban', 'Dodge': 'Urban',
                  'Douglas': 'Rural', 'Faribault': 'Rural', 'Fillmore': 'Rural', 'Freeborn': 'Rural', 'Goodhue': 'Rural',
                  'Grant': 'Rural', 'Hennepin': 'Urban', 'Houston': 'Urban', 'Hubbard': 'Rural', 'Isanti': 'Urban',
                  'Itasca': 'Rural', 'Jackson': 'Rural', 'Kanabec': 'Rural', 'Kandiyohi': 'Rural', 'Kittson': 'Rural',
                  'Koochiching': 'Rural', 'Lac qui Parle': 'Rural', 'Lake': 'Rural', 'Lake of the Woods': 'Rural', 'Le Sueur': 'Rural',
                  'Lincoln': 'Rural', 'Lyon': 'Rural', 'McLeod': 'Rural', 'Mahnomen': 'Rural', 'Marshall': 'Rural',
                  'Martin': 'Rural', 'Meeker': 'Rural', 'Mille Lacs': 'Rural', 'Morrison': 'Rural', 'Mower': 'Rural',
                  'Murray': 'Rural', 'Nicollet': 'Rural', 'Nobles': 'Rural', 'Norman': 'Rural', 'Olmsted': 'Urban',
                  'Otter Tail': 'Rural', 'Pennington': 'Rural', 'Pine': 'Rural', 'Pipestone': 'Rural', 'Polk': 'Urban',
                  'Pope': 'Rural', 'Ramsey': 'Urban', 'Red Lake': 'Rural', 'Redwood': 'Rural', 'Renville': 'Rural',
                  'Rice': 'Rural', 'Rock': 'Rural', 'Roseau': 'Rural', 'St. Louis': 'Urban', 'Scott': 'Urban',
                  'Sherburne': 'Urban', 'Sibley': 'Rural', 'Stearns': 'Urban', 'Steele': 'Rural', 'Stevens': 'Rural',
                  'Swift': 'Rural', 'Todd': 'Rural', 'Traverse': 'Rural', 'Wabasha': 'Urban', 'Wadena': 'Rural',
                  'Waseca': 'Rural', 'Washington': 'Urban', 'Watonwan': 'Rural', 'Wilkin': 'Rural', 'Winona': 'Rural',
                  'Wright': 'Urban', 'Yellow Medicine' : 'Rural'
}

minnesota_data.insert(34, 'OMB County Type', "")

minnesota_data['OMB County Type'] = minnesota_data['county'].map(county_type_omb)
minnesota_data.head()


(1044, 42)
Index(['Sample Population', 'Population Poverty', 'Females Poverty',
       'Females < 5 Poverty', 'Females 5 Poverty', 'Females 6 to 11 Poverty',
       'Females 12 to 14 Poverty', 'Females 15 Poverty',
       'Females 16 to 17 Poverty', 'Females 18 to 24 Poverty',
       'Females 25 to 34 Poverty', 'Females 35 to 44 Poverty',
       'Females 45 to 54 Poverty', 'Females 55 to 64 Poverty',
       'Females 65 to 74 Poverty', 'Females 75 > Poverty', 'Males Poverty',
       'Males < 5 Poverty', 'Males 5 Poverty', 'Males 6 to 11 Poverty',
       'Males 12 to 14 Poverty', 'Males 15 Poverty', 'Males 16 to 17 Poverty',
       'Males 18 to 24 Poverty', 'Males 25 to 34 Poverty',
       'Males 35 to 44 Poverty', 'Males 45 to 54 Poverty',
       'Males 55 to 64 Poverty', 'Males 65 to 74 Poverty',
       'Males 75 > Poverty', 'B01003_001E', 'state code', 'state',
       'county code', 'county', 'year', 'Children in Poverty',
       'Seniors in Poverty', 'Poverty %', 'Female Poverty %', 

,Sample Population,Population Poverty,Females Poverty,Females < 5 Poverty,Females 5 Poverty,Females 6 to 11 Poverty,Females 12 to 14 Poverty,Females 15 Poverty,Females 16 to 17 Poverty,Females 18 to 24 Poverty,...,OMB County Type,county,year,Children in Poverty,Seniors in Poverty,Poverty %,Female Poverty %,Male Poverty %,Childhood Poverty %,Senior Poverty %
1187,16129.0,2185.0,1197.0,133.0,16.0,85.0,37.0,23.0,48.0,124.0,...,Rural,Aitkin,2010,663.0,568.0,13.547027,7.421415,6.125612,4.110608,3.521607
1188,323359.0,20730.0,11736.0,992.0,302.0,1123.0,535.0,124.0,385.0,1314.0,...,Urban,Anoka,2010,7030.0,3320.0,6.410831,3.629403,2.781429,2.174054,1.026723
1189,31644.0,3554.0,2010.0,251.0,69.0,109.0,50.0,21.0,75.0,312.0,...,Rural,Becker,2010,1240.0,877.0,11.231197,6.351915,4.879282,3.918594,2.771457
1190,42043.0,8383.0,4528.0,581.0,98.0,406.0,128.0,88.0,97.0,1021.0,...,Rural,Beltrami,2010,3055.0,892.0,19.939110,10.769926,9.169184,7.266370,2.121637
1191,37411.0,4428.0,2570.0,213.0,100.0,214.0,84.0,25.0,44.0,585.0,...,Urban,Benton,2010,1404.0,550.0,11.836091,6.869637,4.966454,3.752907,1.470156


,Sample Population,Population Poverty,Females Poverty,Females < 5 Poverty,Females 5 Poverty,Females 6 to 11 Poverty,Females 12 to 14 Poverty,Females 15 Poverty,Females 16 to 17 Poverty,Females 18 to 24 Poverty,...,Males 75 > Poverty,B01003_001E,state code,state,county code,OMB County Type,county type,county,year,Poverty %
1187,16129.0,2185.0,1197.0,133.0,16.0,85.0,37.0,23.0,48.0,124.0,...,69.0,16325.0,27,MN,001,Rural,Rural,Aitkin,2010,13.547027
1188,323359.0,20730.0,11736.0,992.0,302.0,1123.0,535.0,124.0,385.0,1314.0,...,243.0,327544.0,27,MN,003,Urban,Urban,Anoka,2010,6.410831
1189,31644.0,3554.0,2010.0,251.0,69.0,109.0,50.0,21.0,75.0,312.0,...,83.0,32274.0,27,MN,005,Rural,Rural,Becker,2010,11.231197
1190,42043.0,8383.0,4528.0,581.0,98.0,406.0,128.0,88.0,97.0,1021.0,...,96.0,43781.0,27,MN,007,Rural,Rural,Beltrami,2010,19.939110
1191,37411.0,4428.0,2570.0,213.0,100.0,214.0,84.0,25.0,44.0,585.0,...,50.0,38148.0,27,MN,009,Urban,Urban,Benton,2010,11.836091


In [31]:
# minnesota_data['Females Test Count'] = minnesota_data['Females < 5 Poverty'] + minnesota_data['Females 5 Poverty'] \
#     + minnesota_data['Females 6 to 11 Poverty'] + minnesota_data['Females 12 to 14 Poverty'] \
#     + minnesota_data['Females 15 Poverty'] + minnesota_data['Females 16 to 17 Poverty'] \
#     + minnesota_data['Females 18 to 24 Poverty'] + minnesota_data['Females 25 to 34 Poverty'] \
#     + minnesota_data['Females 35 to 44 Poverty'] + minnesota_data['Females 45 to 54 Poverty'] \
#     + minnesota_data['Females 55 to 64 Poverty'] + minnesota_data['Females 65 to 74 Poverty'] \
#     + minnesota_data['Females 75 > Poverty']




minnesota_data.head()




,Sample Population,Population Poverty,Females Poverty,Females < 5 Poverty,Females 5 Poverty,Females 6 to 11 Poverty,Females 12 to 14 Poverty,Females 15 Poverty,Females 16 to 17 Poverty,Females 18 to 24 Poverty,...,OMB County Type,county,year,Children in Poverty,Seniors in Poverty,Poverty %,Female Poverty %,Male Poverty %,Childhood Poverty %,Senior Poverty %
1187,16129.0,2185.0,1197.0,133.0,16.0,85.0,37.0,23.0,48.0,124.0,...,Rural,Aitkin,2010,663.0,568.0,13.547027,7.421415,6.125612,4.110608,3.521607
1188,323359.0,20730.0,11736.0,992.0,302.0,1123.0,535.0,124.0,385.0,1314.0,...,Urban,Anoka,2010,7030.0,3320.0,6.410831,3.629403,2.781429,2.174054,1.026723
1189,31644.0,3554.0,2010.0,251.0,69.0,109.0,50.0,21.0,75.0,312.0,...,Rural,Becker,2010,1240.0,877.0,11.231197,6.351915,4.879282,3.918594,2.771457
1190,42043.0,8383.0,4528.0,581.0,98.0,406.0,128.0,88.0,97.0,1021.0,...,Rural,Beltrami,2010,3055.0,892.0,19.939110,10.769926,9.169184,7.266370,2.121637
1191,37411.0,4428.0,2570.0,213.0,100.0,214.0,84.0,25.0,44.0,585.0,...,Urban,Benton,2010,1404.0,550.0,11.836091,6.869637,4.966454,3.752907,1.470156


In [32]:
minnesota_data.head()

,Sample Population,Population Poverty,Females Poverty,Females < 5 Poverty,Females 5 Poverty,Females 6 to 11 Poverty,Females 12 to 14 Poverty,Females 15 Poverty,Females 16 to 17 Poverty,Females 18 to 24 Poverty,...,OMB County Type,county,year,Children in Poverty,Seniors in Poverty,Poverty %,Female Poverty %,Male Poverty %,Childhood Poverty %,Senior Poverty %
1187,16129.0,2185.0,1197.0,133.0,16.0,85.0,37.0,23.0,48.0,124.0,...,Rural,Aitkin,2010,663.0,568.0,13.547027,7.421415,6.125612,4.110608,3.521607
1188,323359.0,20730.0,11736.0,992.0,302.0,1123.0,535.0,124.0,385.0,1314.0,...,Urban,Anoka,2010,7030.0,3320.0,6.410831,3.629403,2.781429,2.174054,1.026723
1189,31644.0,3554.0,2010.0,251.0,69.0,109.0,50.0,21.0,75.0,312.0,...,Rural,Becker,2010,1240.0,877.0,11.231197,6.351915,4.879282,3.918594,2.771457
1190,42043.0,8383.0,4528.0,581.0,98.0,406.0,128.0,88.0,97.0,1021.0,...,Rural,Beltrami,2010,3055.0,892.0,19.939110,10.769926,9.169184,7.266370,2.121637
1191,37411.0,4428.0,2570.0,213.0,100.0,214.0,84.0,25.0,44.0,585.0,...,Urban,Benton,2010,1404.0,550.0,11.836091,6.869637,4.966454,3.752907,1.470156


In [33]:
minnesota_data.columns

Index(['Sample Population', 'Population Poverty', 'Females Poverty',
       'Females < 5 Poverty', 'Females 5 Poverty', 'Females 6 to 11 Poverty',
       'Females 12 to 14 Poverty', 'Females 15 Poverty',
       'Females 16 to 17 Poverty', 'Females 18 to 24 Poverty',
       'Females 25 to 34 Poverty', 'Females 35 to 44 Poverty',
       'Females 45 to 54 Poverty', 'Females 55 to 64 Poverty',
       'Females 65 to 74 Poverty', 'Females 75 > Poverty', 'Males Poverty',
       'Males < 5 Poverty', 'Males 5 Poverty', 'Males 6 to 11 Poverty',
       'Males 12 to 14 Poverty', 'Males 15 Poverty', 'Males 16 to 17 Poverty',
       'Males 18 to 24 Poverty', 'Males 25 to 34 Poverty',
       'Males 35 to 44 Poverty', 'Males 45 to 54 Poverty',
       'Males 55 to 64 Poverty', 'Males 65 to 74 Poverty',
       'Males 75 > Poverty', 'B01003_001E', 'state code', 'state',
       'county code', 'OMB County Type', 'county', 'year',
       'Children in Poverty', 'Seniors in Poverty', 'Poverty %',
       'Fema

In [34]:
#filtering 
minnesota_data.groupby("year")["Poverty %"].describe()

,count,mean,std,min,25%,50%,75%,max
year,,,,,,,,
2010,87.0,10.898861,3.402141,4.653798,8.889552,10.744783,12.433949,23.601066
2011,87.0,11.384684,3.599338,5.012521,9.041550,11.242324,12.836302,26.444111
2012,87.0,11.563574,3.666374,5.168681,9.036022,11.452958,13.028723,27.164401
2013,87.0,11.684700,3.505883,4.961699,9.315323,11.607496,13.346301,26.196660
2014,87.0,11.757248,3.646107,4.355141,9.455015,11.401833,13.493838,26.734505
2015,87.0,11.669400,3.454009,4.401726,9.312099,11.408104,13.556139,24.926145
2016,87.0,11.253355,3.302798,4.112793,9.059472,11.502467,13.035114,22.405486
2017,87.0,11.038760,3.360757,4.020604,8.842783,11.304488,12.535072,22.668153
2018,87.0,10.752444,3.257427,3.961521,8.649929,10.582895,12.760972,20.874419


In [35]:
minnesota_data.head()

,Sample Population,Population Poverty,Females Poverty,Females < 5 Poverty,Females 5 Poverty,Females 6 to 11 Poverty,Females 12 to 14 Poverty,Females 15 Poverty,Females 16 to 17 Poverty,Females 18 to 24 Poverty,...,OMB County Type,county,year,Children in Poverty,Seniors in Poverty,Poverty %,Female Poverty %,Male Poverty %,Childhood Poverty %,Senior Poverty %
1187,16129.0,2185.0,1197.0,133.0,16.0,85.0,37.0,23.0,48.0,124.0,...,Rural,Aitkin,2010,663.0,568.0,13.547027,7.421415,6.125612,4.110608,3.521607
1188,323359.0,20730.0,11736.0,992.0,302.0,1123.0,535.0,124.0,385.0,1314.0,...,Urban,Anoka,2010,7030.0,3320.0,6.410831,3.629403,2.781429,2.174054,1.026723
1189,31644.0,3554.0,2010.0,251.0,69.0,109.0,50.0,21.0,75.0,312.0,...,Rural,Becker,2010,1240.0,877.0,11.231197,6.351915,4.879282,3.918594,2.771457
1190,42043.0,8383.0,4528.0,581.0,98.0,406.0,128.0,88.0,97.0,1021.0,...,Rural,Beltrami,2010,3055.0,892.0,19.939110,10.769926,9.169184,7.266370,2.121637
1191,37411.0,4428.0,2570.0,213.0,100.0,214.0,84.0,25.0,44.0,585.0,...,Urban,Benton,2010,1404.0,550.0,11.836091,6.869637,4.966454,3.752907,1.470156


In [36]:
# Specify the file path where you want to save the CSV file
csv_file_path = 'Resources/minnesota_poverty_data.csv'

# Output the DataFrame to a CSV file
minnesota_data.to_csv(csv_file_path, index=False)

# Display a message indicating success
print(f"DataFrame has been successfully saved to {csv_file_path}")

DataFrame has been successfully saved to Resources/minnesota_poverty_data.csv
